### Convolutional Neural Network Testing
All credit for the NN goes to Ian Flores.  
This notebook is meant for testing the NN with only two classes:  
x species or not x species

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import os
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

### Deep Convolution Neural Network
batch_size = 79  
patch_size = 2  
depth = 5  
num_hidden = 100

### Structure of the Network
Input Layer: image  
Layer 1: First Convolution Layer  
Pooling: RELU  
Layer 2: Second Convolution Layer  
Pooling: RELU  
Layer 5: Reshape  
Layer 6: RELU

In [56]:
def convNN(pickle_file):

    name = pickle_file.split('/')[-1].split('.')[0].split('_')[-1]
    
    print("Using binary {} dataset\n".format(name))
    
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        train_dataset = save['train_dataset']
        train_labels = save['train_labels']
        valid_dataset = save['valid_dataset']
        valid_labels = save['valid_labels']
        test_dataset = save['test_dataset']
        test_labels = save['test_labels']
        del save  # hint to help gc free up memory
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)

    #print("cuestion!!:", valid_labels)
    
    image_size = 100
    num_labels = 2
    num_channels = 1 # grayscale
    
    # change later
    def reformat(dataset, labels):
        dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
        labels = np.asarray(labels, dtype=np.float32)
        return dataset, labels
    train_dataset, train_labels = reformat(train_dataset, train_labels)
    valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
    test_dataset, test_labels = reformat(test_dataset, test_labels)
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    print(test_labels.shape)
    
    def accuracy(predictions, labels):
        acc = tf.metrics.accuracy(labels,predictions)
        return acc.eval()
    
    cms = []

    batch_size = 100
    patch_size = 2
    depth = 5
    num_hidden = 100

    graph = tf.Graph()

    with graph.as_default():

        # Input data.
        tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(None))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
        layer1_biases = tf.Variable(tf.zeros([depth]))
        layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
        layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
        layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

        tf_test_labels = tf.constant(test_labels)

        # Model.
        conv = tf.nn.conv2d(tf_train_dataset, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [-1, shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        logits = tf.matmul(hidden, layer4_weights) + layer4_biases

        # Training computation.
        # logits = model(tf_train_dataset)
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(logits)
        test_prediction = tf.nn.softmax(logits)
    
    num_steps = 1001

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized NN with {} dataset".format(name))
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size)]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            vps = valid_prediction.eval(feed_dict={tf_train_dataset: valid_dataset})
            pred = np.argmax(vps)
            print("Valid predictions: ", pred.eval())
#             print("Valid preobability scores: ", valid_prediction.eval())
#             print("Valid labels: ", list(valid_labels))
            if (step % 50 == 0):
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))

        preds = test_prediction.eval()
        preds = tf.reshape(preds, [-1])
        tf_test_labels = tf.reshape(tf_test_labels, [-1])
        cm = tf.confusion_matrix(tf_test_labels, preds, 2)
        cm = cm.eval()
        preds = preds.eval()
        lbls = tf_test_labels.eval()
        
        cms.append(cm)
        
        print('Test accuracy of %s: %.1f%%' % (name, accuracy(test_prediction.eval(), test_labels)))
    print("\n\n\n")
    return cms

In [60]:
pickle_file = '../dataset/arbimon_bin_datasets/arbimon_bin_Basileuterus bivittatus.pickle'
name = pickle_file.split('/')[-1].split('.')[0].split('_')[-1]
    
print("Using binary {} dataset\n".format(name))

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

#print("cuestion!!:", valid_labels)

image_size = 100
num_labels = 2
num_channels = 1 # grayscale

# change later
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = np.asarray(labels, dtype=np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print(test_labels.shape)

def accuracy(predictions, labels):
    acc = tf.metrics.accuracy(labels,predictions)
    return acc.eval()

cms = []

batch_size = 100
patch_size = 2
depth = 5
num_hidden = 100

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    tf_test_labels = tf.constant(test_labels)

    # Model.
    conv = tf.nn.conv2d(tf_train_dataset, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    logits = tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    # logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(logits)

num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized NN with {} dataset".format(name))
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        vps = valid_prediction.eval(feed_dict={tf_train_dataset: valid_dataset})
        pred = np.argmax(vps)
        print("Valid predictions: ", pred.eval())
#             print("Valid preobability scores: ", valid_prediction.eval())
#             print("Valid labels: ", list(valid_labels))
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))

    preds = test_prediction.eval()
    preds = tf.reshape(preds, [-1])
    tf_test_labels = tf.reshape(tf_test_labels, [-1])
    cm = tf.confusion_matrix(tf_test_labels, preds, 2)
    cm = cm.eval()
    preds = preds.eval()
    lbls = tf_test_labels.eval()

    cms.append(cm)

    print('Test accuracy of %s: %.1f%%' % (name, accuracy(test_prediction.eval(), test_labels)))
print("\n\n\n")

Using binary Basileuterus bivittatus dataset

Training set (615, 100, 100) (615,)
Validation set (246, 100, 100) (246,)
Test set (369, 100, 100) (369,)
Training set (615, 100, 100, 1) (615,)
Validation set (246, 100, 100, 1) (246,)
Test set (369, 100, 100, 1) (369,)
(369,)
Initialized NN with Basileuterus bivittatus dataset


InvalidArgumentError: logits and labels must be same size: logits_size=[100,2] labels_size=[1,100]
	 [[Node: SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape_1, Reshape_2)]]

Caused by op 'SoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/asvnpr/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).s

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized NN with {} dataset".format(name))
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        vps = valid_prediction.eval(feed_dict={tf_train_dataset: valid_dataset})
        pred = np.argmax(vps)
        print("Valid predictions: ", pred.eval())
#             print("Valid preobability scores: ", valid_prediction.eval())
#             print("Valid labels: ", list(valid_labels))
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))

    preds = test_prediction.eval()
    preds = tf.reshape(preds, [-1])
    tf_test_labels = tf.reshape(tf_test_labels, [-1])
    cm = tf.confusion_matrix(tf_test_labels, preds, 2)
    cm = cm.eval()
    preds = preds.eval()
    lbls = tf_test_labels.eval()

    cms.append(cm)

    print('Test accuracy of %s: %.1f%%' % (name, accuracy(test_prediction.eval(), test_labels)))
print("\n\n\n")

In [57]:
datasets = '../dataset/arbimon_bin_datasets'

cms = []
for dataset in os.listdir(datasets):
    cms = convNN(datasets+'/'+dataset)

Using binary Microcerculus marginatus dataset

Training set (492, 100, 100) (492,)
Validation set (196, 100, 100) (196,)
Test set (295, 100, 100) (295,)
Training set (492, 100, 100, 1) (492,)
Validation set (196, 100, 100, 1) (196,)
Test set (295, 100, 100, 1) (295,)
(295,)
Initialized NN with Microcerculus marginatus dataset


InvalidArgumentError: logits and labels must be same size: logits_size=[100,2] labels_size=[1,100]
	 [[Node: SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape_1, Reshape_2)]]

Caused by op 'SoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/asvnpr/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).s

In [49]:
import matplotlib.pyplot as plt
import itertools
    
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

for cm in cms:
    # Compute confusion matrix
    cnf_matrix = cm
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure(num=1, figsize=(12, 12), dpi=80)
    plot_confusion_matrix(cnf_matrix, classes=list(range(2)),title='Confusion matrix, without normalization')
    #plt.figure(num=1, figsize=(8, 6), dpi=80)

    # Plot normalized confusion matrix
    plt.figure(num=2, figsize=(12, 12), dpi=80)
    plot_confusion_matrix(cnf_matrix, classes=list(range(2)), normalize=True, title='Normalized confusion matrix')
    #plt.figure(num=2, figsize=(8, 6), dpi=80)

    plt.show()